In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')


import json
from datetime import datetime
from datetime import timedelta
import pandas as pd
import time

In [ ]:
def StreamingStat2(session):
    from_to = {"00":"01","01":"02","02":"03","03":"04","04":"05","05":"06","06":"07","07":"08","08":"09","09":"10","10":"11",
    "11":"12","12":"13","13":"14","14":"15","15":"16","16":"17","17":"18","18":"19","19":"20","20":"21","21":"22","22":"23","23":"00"}
    #Process Remaining data
    #max_day = get the max day in stat2
    last_day = pd.DataFrame(list(session.execute('SELECT max(day) as last_day FROM "statistics_2";')))['last_day'][0]

    if last_day != None: #stat2 is not empty
        last_day = last_day.split(" ")[0]
        #get the the timestamp for the last processed record
        last_record_ts = pd.DataFrame(list(session.execute(f'SELECT max(LPRD) as last_record FROM "statistics_2" where \
            day=\'{last_day}\' allow filtering;')))['last_record'][0]
        #get the day that follow the current date 
        date_1 = datetime.strptime(last_day, "%Y-%m-%d")
        end_date = date_1 + timedelta(days=1)
        #get the data ordrer by timestamp
        data = pd.DataFrame(list(session.execute(f"SELECT * FROM Event where \
            timestamp > '{last_record_ts}' and  timestamp < '{end_date}' order by timestamp allow filtering;")))

        if(data.shape[0]>0):#if there is remaining data
            #get the hour of transaction for the first record

            data['from'] = data['timestamp'].apply(lambda x: (str(x).split(" ")[1]).split(":")[0])

            #get the first start_hour
            from_0 = data.loc[0,"from"]
            
            i=0 # to iterate through the data
            #check if the start_hour exists in statistics_2

            check_query = pd.DataFrame(list(session.execute(f'SELECT id FROM "statistics_2" where day={last_day} and from={from_0} allow filtering;')))['id']

            if(check_query.shape[0] > 0):

                new_hour = False

                old_stats = pd.DataFrame(list(session.execute(f'SELECT id, tram_A,tram_B,tram_C FROM "statistics_2" where day={last_day} and from={from_0} allow filtering;')))

                line_A = json.loads(old_stats.loc[0,"tram_A"])
                line_B = json.loads(old_stats.loc[0,"tram_B"])
                line_C = json.loads(old_stats.loc[0,"tram_C"])
                
                while(i < data.shape[0]):

                    sh = data.loc[i,"from"]
                    
                    if (sh != from_0):
                        
                        new_hour = True

                        session.execute(f"UPDATE statistics_2 SET \
                        tram_A ={line_A},tram_B= {line_B},tram_C='{line_C}',LPRD='{data.loc[i,'timestamp']}'\
                            WHERE id = {old_stats.loc[0,'id']};")

                        del line_A
                        del line_B
                        del line_C

                        break
                    
                    #updating statistics
                    update(data,i,line_A,line_B,line_C)                      

                    if(i== (data.shape[0] - 1)):
                        
                        session.execute(f"UPDATE statistics_2 SET \
                        tram_A ={line_A},tram_B= {line_B},tram_C='{line_C}',LPRD='{data.loc[i,'timestamp']}'\
                            WHERE id = {old_stats.loc[0,'id']};")
                        
                        del line_A
                        del line_B
                        del line_C

                    i+=1

                j = old_stats.loc[0,'id'] + 1

                if(new_hour):


                    
                    from_0 = data.loc[i,"from"]



                    line_A = {"users":0}
                    line_B = {"users":0}
                    line_C = {"users":0}

                    while(i < data.shape[0]):
                        
                        if(from_0 == data.loc[i,"from"]):

                                update(data,i,line_A,line_B,line_C)
                                
                                if(i== (data.shape[0]-1)):

                                    session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{last_day}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i,'timestamp']}');")

                                    j+=1 
                            
                                                     
                                i+=1
                        
                        else:
                            

                            session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{last_day}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i-1,'timestamp']}');")
                            
                            j+=1 

                            del line_A
                            del line_B
                            del line_C
                            line_A = {"users":0}
                            line_B = {"users":0}
                            line_C = {"users":0}


                            from_0 = data.loc[i,'from']
            else:#it's a new hour




                    from_0 = data.loc[i,"from"]

                    previous_hour = list(from_to.keys())[list(from_to.values()).index(from_0)]
                    
                    #get the index of there previous record of the hour before the current hour

                    

                    old_id = pd.DataFrame(list(session.execute(f'SELECT id FROM "statistics_2" where day={last_day} and from={previous_hour} allow filtering;')))['id'][0]

                    j = old_id + 1



                    line_A = {"users":0}
                    line_B = {"users":0}
                    line_C = {"users":0}

                    while(i < data.shape[0]):
                        
                        if(from_0 == data.loc[i,"from"]):

                                update(data,i,line_A,line_B,line_C)
                                
                                if(i==data.shape[0]-1):

                                    session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{last_day}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i,'timestamp']}');")
                                    j+=1
                                                     
                                i+=1
                        
                        else:
                            session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j+1}, '{last_day}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i-1,'timestamp']}');") 
                            
                            j+=1

                            del line_A
                            del line_B
                            del line_C
                            line_A = {"users":0}
                            line_B = {"users":0}
                            line_C = {"users":0}


                            from_0 = data.loc[i,'from']

    ## Process data from the beginning

    else:
        dates = pd.DataFrame(list(session.execute(f"SELECT toDate(timestamp) as date\
         FROM Event;")))['date'].unique()


        dates.sort()
    
        for date in dates:
            

            date_1 = (datetime.strptime(str(date), "%Y-%m-%d"))
            end_date = date_1 + timedelta(days=1)
            end_date = str(end_date.date())
            date = str(date)
            print(f"date --> {date}")

            data =  pd.DataFrame(list(session.execute(f"SELECT * FROM Event where \
            timestamp >= '{date}' and  timestamp < '{end_date}' allow filtering;")))

            data['from'] = data['timestamp'].apply(lambda x: (str(x).split(" ")[1]).split(":")[0])

            i=0
            from_0 = data.loc[i,"from"]

            line_A = {"users":0}
            line_B = {"users":0}
            line_C = {"users":0}

            j=1

            while(i < data.shape[0]):
                        
                    if(from_0 == data.loc[i,"from"]):

                        update(data,i,line_A,line_B,line_C)
                                
                        if(i== (data.shape[0]-1)):

                            session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{date}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i,'timestamp']}');")

                            j+=1   
                                                     
                        i+=1
                        
                    else:

                        session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{date}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i-1,'timestamp']}');") 
                        
                        j+=1

                        del line_A
                        del line_B
                        del line_C
                        line_A = {"users":0}
                        line_B = {"users":0}
                        line_C = {"users":0}


                        from_0 = data.loc[i,'from']


    ## Process streaming data
    while(True):

        time.sleep(5)

        last_timestamp_processed = pd.DataFrame(list(session.execute(f'SELECT max(LPRD) as last_record FROM "statistics_2";')))['last_record'][0]
        
        data = pd.DataFrame(list(session.execute(f"SELECT * FROM Event where \
            timestamp > '{last_timestamp_processed}' order by timestamp allow filtering;")))

        data['from'] = data['timestamp'].apply(lambda x: (str(x).split(" ")[1]).split(":")[0])

        
        

        if(data.shape[0] > 0): #if there is new data

            date = str(data.loc[0,'timestamp']).split(" ")[0]
            
            i=0 # to iterate through the data
            #check if the start_hour exists in statistics_2

            from_0 = data.loc[0,'from']

            check_query = pd.DataFrame(list(session.execute(f"SELECT id FROM statistics_2 where day='{date}' and from='{from_0}' allow filtering;")))['id']

            if(check_query.shape[0] > 0):

                new_hour = False

                 

                old_stats = pd.DataFrame(list(session.execute(f"SELECT id, tram_A,tram_B,tram_C FROM statistics_2 where day='{date}' and from='{from_0}' allow filtering;")))

                line_A = json.loads(old_stats.loc[0,"tram_A"])
                line_B = json.loads(old_stats.loc[0,"tram_B"])
                line_C = json.loads(old_stats.loc[0,"tram_C"])
                
                while(i < data.shape[0]):

                    sh = data.loc[i,"from"]
                    
                    if (sh != from_0):
                        
                        new_hour = True

                        session.execute(f"UPDATE statistics_2 SET \
                        tram_A ={line_A},tram_B= {line_B},tram_C='{line_C}',LPRD={data.loc[i-1,'timestamp']}\
                            WHERE id = {old_stats.loc[0,'id']};")

                        del line_A
                        del line_B
                        del line_C

                        break

                    update(data,i,line_A,line_B,line_C)                      

                    if(i== (data.shape[0] - 1)):
                        
                        session.execute(f"UPDATE statistics_2 SET \
                        tram_A ={line_A},tram_B= {line_B},tram_C='{line_C}',LPRD='{data.loc[i,'timestamp']}'\
                            WHERE id = {old_stats.loc[0,'id']};")
                        
                        del line_A
                        del line_B
                        del line_C

                    i+=1

                if(new_hour):

                    j= int(old_stats.loc[0,'id']) + 1
                    
                    from_0 = data.loc[i,"from"]
                    
                    date = str(data.loc[i,'timestamp']).split(" ")[0]

                    line_A = {"users":0}
                    line_B = {"users":0}
                    line_C = {"users":0}

                    while(i < data.shape[0]):
                        
                        if(from_0 == data.loc[i,"from"]):

                                update(data,i,line_A,line_B,line_C)
                                
                                if(i== (data.shape[0]-1)):

                                    session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{date}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i,'timestamp']}');")

                                    j+=1  
                                                     
                                i+=1
                        
                        else:

                        
                            session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({j}, '{date}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i-1,'timestamp']}');") 
                            
                            j+=1

                            if(data.loc[i,"from"] == "00"):
                                
                                    date = str(data.loc[i,'timestamp']).split(" ")[0]

                            del line_A
                            del line_B
                            del line_C
                            line_A = {"users":0}
                            line_B = {"users":0}
                            line_C = {"users":0}


                            from_0 = data.loc[i,'from']
            else:#it's a new hour

                    
                    from_0 = data.loc[i,"from"]

                    previous_hour = list(from_to.keys())[list(from_to.values()).index(from_0)]
                    
                    #get the index of there previous record of the hour before the current hour

                    

                    old_id = pd.DataFrame(list(session.execute(f"SELECT id FROM statistics_2 where day='{last_day}' and from='{previous_hour}' allow filtering;")))['id'][0]

                    j = old_id + 1
                    line_A = {"users":0}
                    line_B = {"users":0}
                    line_C = {"users":0}

                    while(i < data.shape[0]):
                        
                        if(from_0 == data.loc[i,"from"]):

                                update(data,i,line_A,line_B,line_C)
                                
                                if(i==data.shape[0]-1):

                                    session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({index}, '{last_day}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i,'timestamp']}');")
                            
                                                     
                                i+=1
                        
                        else:
                            session.execute(f"INSERT INTO statistics_2(id,day,end_hour,start_hour,tram_A ,tram_B ,tram_C,LPRD )\
            VALUES ({index}, '{last_day}','{from_to[from_0]}','{from_0}',{line_A},{line_B},{line_C},'{data.loc[i-1,'timestamp']}');") 

                            if(data.loc[i,"from"] == "00"):
                                
                                    date = str(data.loc[i,'timestamp']).split(" ")[0]

                            del line_A
                            del line_B
                            del line_C
                            line_A = {"users":0}
                            line_B = {"users":0}
                            line_C = {"users":0}


                            from_0 = data.loc[i,'from']        


In [ ]:
############ Process Remaining data
#->max_day = get the max day in stat2 
#->max_ts_day = max timestamp for the current day from stat2
#->data = data from event table where timestamp > max_ts_day and timestamp <next_day 
#->sort data by timestamp
#->start_hour = data[0]
#->i=0   old: 5,5,5  new: 5, 5, 5, 5, 6
#->if(start_hour already exists):#updating
#   ->stats = stats from stats2 where start_hour =new_data[start_hour]
#   ->while(i < len(data)):
#           ->sh = data[hour]
#           ->if(sh != start_hour):
#               ->new_hour = True
#               ->update(id_event,date,start_hour,stats,data.loc[i,timestamp])
#               ->break
#           ->{users++, data.loc[i,start_station]++}
#           ->i++
#           ->if(i==len(data.shape[0]-1)):
#             
#                   ->update(id_event,date,start_hour,stats,data.loc[i,timestamp])
#   ->if(start_hour == True):
#       ->start_hour = data[i]
#       ->stats = {users=0}
#       ->while(i< len(data.shape[0])):
#           ->if(start_hour == z.loc[i,hour]): 
#               ->{users++, z.loc[i,start_station]++}
#               ->if(i==len(data)-1):
#                   ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#               ->i++
#           ->else:
#               ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#               ->stats={users=0}
#               ->start_hour=z.loc[i,hour]
#
#->else:  #new hour 
#   ->while(i< len(data.shape[0])):
#       ->if(start_hour == z.loc[i,hour]): 
#           ->{users++, z.loc[i,start_station]++}
#           ->if(i==len(data)-1):
#               ->insert(id_event,date,start_hour,stats,data.loc[i,timestamp])
#           ->i++
#       ->else:
#           ->insert(id_event,date,start_hour,stats,data.loc[i,timestamp])
#           ->stats={users=0}
#           ->start_hour=data.loc[i,hour]



############### Process data from beginning
# ->for each date:
#   ->get data from event for the current date
#   ->sort data by timestamp
#   ->start_hour = data[0]
#   ->stats = {users=0}
#   ->i=0
#   ->while(i< len(data.shape[0])):
#       ->if(start_hour == z.loc[i,hour]): 
#           ->{users++, z.loc[i,start_station]++}
#           ->if(i==len(data)-1):
#               ->insert(id_event,date,start_hour,stats,data.loc[i,timestamp])
#           ->i++
#       ->else:
#           ->insert(id_event,date,start_hour,stats,data.loc[i,timestamp])
#           ->stats={users=0}
#           ->start_hour=data.loc[i,hour]
#
#############Process Streaming data
#
#->while(True):
#   ->sleep(5)
#   ->lts = get the max timestamp LSTP from stat2
#   ->data = check if there is new data in event where timestamp > lts
#   ->sort data by timestamp
#   ->start_hour = data[0]
#   ->i=0   old: 5,5,5  new: 5, 5, 5, 5, 6
#   ->if(start_hour already exists):#updating
#       ->stats = stats from stats2 where start_hour =new_data[start_hour]
#       ->while(i < len(data)):
#           ->sh = data[hour]
#           ->if(sh != start_hour):
#               ->new_hour = True
#               ->break
#           ->{users++, data.loc[i,start_station]++}
#           ->i++
#           ->if(i==len(data.shape[0]-1)):
#        
#                   ->update(id_event,date,start_hour,stats,data.loc[i,timestamp])
#       ->if(start_hour == True):
#       ->start_hour = data[i]
#       ->stats = {users=0}
#       ->while(i< len(data.shape[0])):
#           ->if(start_hour == z.loc[i,hour]): 
#               ->{users++, z.loc[i,start_station]++}
#               ->if(i==len(data)-1):
#                   ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#               ->i++
#       ->else:
#           ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#           ->stats={users=0}
#           ->start_hour=z.loc[i,hour]
#
#
#
#
#
#
#
#

In [1]:
def update(data,index,A,B,C):

    if(data.loc[index,"line"]== "A"):

        A["users"] = A["users"] + 1

        if(data.loc[index,"start_station"] in A):
            A[data.loc[index,"start_station"]] = A[data.loc[index,"start_station"]] + 1
        else:
            A[data.loc[index,"start_station"]] = 1

    elif(data.loc[index,"line"]== "B"):
            if(data.loc[index,"start_station"] in B):
                B[data.loc[index,"start_station"]] = B[data.loc[index,"start_station"]] + 1
            else:
                B[data.loc[index,"start_station"]] = 1                       
    else:       
            if(data.loc[index,"start_station"] in C):
                C[data.loc[index,"start_station"]] = C[data.loc[index,"start_station"]] + 1
            else:
                C[data.loc[index,"start_station"]] = 1

In [2]:
from_to = {"00":"01","01":"02","02":"03","03":"04","04":"05","05":"06","06":"07","07":"08","08":"09","09":"10","10":"11",
    "11":"12","12":"13","13":"14","14":"15","15":"16","16":"17","17":"18","18":"19","19":"20","20":"21","21":"22","22":"23","23":"00"}
previous_hour = list(from_to.keys())[list(from_to.values()).index("00")]
print(previous_hour)

23
